# Homework 5 - Visit the Wikipedia hyperlinks graph!

# RQ1

In [13]:
import pandas as pd
import json
import pickle
from tqdm import tqdm

from collections import defaultdict
from heapq import *
import numpy as np
import collections
from functions import *

Let's start downloading <a href="https://drive.google.com/file/d/1ghPJ4g6XMCUDFQ2JPqAVveLyytG8gBfL/view">Wikicat hyperlink graph</a> . 

It is a reduced version of the one we can find on SNAP. 

Every row is an <b>edge</b>. The two elements of each row are the <b>nodes</b>, in particular the first is the <b> source</b>, the second represents the <b>destination</b>.

So, our first goal is open and read the file with python, and split each line with the new line charachter.
Then we take all the <i>source nodes</i> for each row, and we put them as keys into our <b>graph</b> dictionary. The values will be all the correspondent destination nodes.

But we have not done! Infact our scope is to analyze the graph, in particular discovering the following informations:

* If it is direct or not;

* The number of nodes;

* The number of edges;

* The average node degree. Is the graph dense?

To do this we want that our dictionary has as keys <u>all the nodes</u>, sources and destinations and for now we have just the first ones. So we add as new keys all the nodes that are just destinations, putting as values empty lists.


Now we have the dictionary with all the nodes of our graph as keys, and as values there are all the eventual destinations!

In [2]:
F = open('wiki-topcats-reduced.txt','r') 
all_rows = F.read().split('\n')

graph = {}
for row in all_rows:
    row = row.split("\t")
    if row[0] not in graph:
        try:
            graph[row[0]] = [row[1]]
        except:
            pass
    else:
        graph[row[0]].append(row[1])
    
    

In [3]:
lista = []
for l in graph.values():
    lista+= l
    

In [4]:
for node in lista:
    if node not in graph:
        graph[node] = []
    else:
        pass

So, what can we say?

* We are in a case of <b>page ranking</b>. So for definition we have nodes representing sources and destinations, with edges with a particular direction. In other words, our graph has a set of edges which are <i>ordered pairs</i> of nodes, and for the graph theory we have a <b>directed graph</b>.


* The number of nodes is <u>461193</u>. It's just the number of keys of our dictionary.


* The number of edges is <u>2645247</u> and it's computed looking at all the lenghts of the <b>adjacency list</b>.


* In graph theory, the <b>degree</b> (or <i>valency</i>) of a vertex of a graph is the number of edges incident to the vertex. We need the <b>average node degree</b>, so we compute the ratio between number of edges and number of nodes. It results <u>5.735661642739591</u>.

#### Number of nodes

In [5]:
V = list(graph.keys())
n_nodes = len(V)
n_nodes

461193

#### Number of edges

In [6]:
n_edges = 0
for l in graph.values():
    n_edges += len(l)
n_edges    

2645247

#### Avarage node degree

In [7]:
avg_degree = n_edges/n_nodes
avg_degree

5.735661642739591

# RQ2

Let's start creating a dictionary called <b>categories</b> where for every category taken from <i>wiki-topcats-categories.txt</i> file, we have the list of all its articles. But attention! We must take into account all the categories that has a number of articles greater than <b>3500</b>. So we filter our dictionary considering the categories with more that 3500 articles. Another, we take just the articles that are also in our <b>graph</b>.


We create also a dictionary called <b>inv_dic</b> that shows for every node (article), a set of all the categories associated. 


In [8]:
C = open('wiki-topcats-categories.txt','r') 

In [9]:
categories = {}
for line in C.readlines():
    l = line.split(' ')
    cat = l[0].replace("Category:","").replace(";", "")
    art = l[1:]
    art[-1] = art[-1].replace("\n","")
    if len(art) >= 3500:
        categories[cat]= set(art).intersection(set(V))



In [10]:
all_set = categories.values()
all_nodes = []
for s in all_set:
    all_nodes += s
inv_dic = {}
for node in all_nodes:
    for cat in categories:
        if node in categories[cat] and node not in inv_dic:
            inv_dic[node] = [cat]
        elif node in categories[cat] and node in inv_dic and cat not in inv_dic[node]:
            inv_dic[node].append(cat)
        else:
            pass


## Block Ranking 

<img src="https://upload.wikimedia.org/wikipedia/commons/5/5d/Breadth-First-Search-Algorithm.gif">

Our scope now is, to take in input a category $C_0 = \{article_1, article_2, \dots \}$. Then we want to rank all of the nodes according to the following criterion:

Obtain a <b>block-ranking</b>, where the blocks are represented by the categories.
The first category of the rank, $C_0$, always corresponds to the input category. The order of the remaining categories is given by: $$distance(C_0, C_1) = median(ShortestPath(C_0, C_i))$$

How we do that? At first we create the functions we need. They all are in the <i>functions.py</i> file for convention. 

The first function we write is the <b> ShortestPath</b> that returns.......

In [11]:
input_category = input()


Year_of_birth_unknown


In [16]:
block_ranking = calculateDistance(input_category, graph, categories, inv_dic)

NameError: name 'tqdm' is not defined

## Sorting

Once we obtain the <i>block ranking vector</i>, we want to sort the nodes in each category. The way we should sort them is the following...

 We have to compute the subgraph induced by $C_0$. Then, for each node, we compute the sum of the weigths of the <b>in-edges</b>. The nodes will be ordered by this score.
 The following image explains how to do it for each step.
 
 In other words, we have to consider a category, and for that category we must compute for each node the number of in-edges, but considering just those that have the source of the same category! For example, in the first image, the B node of the category "0" has got 2 in-edges, but only one is from a node of the same category.

<img src="https://raw.githubusercontent.com/CriMenghini/ADM-2018/master/Homework_5/imgs/algorithm.PNG">

In [ ]:
filtered_nodes = set(all_nodes)



In [ ]:
cat = 'Year_of_birth_unknown'

In [ ]:
n_cat = categories[cat]

In [ ]:
d = {}
for n1 in n_cat:
    count = 0
    for n2 in graph:
        if n1 in graph[n2] and n2 in n_cat:
            count += 1
    d[n1] = count
            
    
            
    

In [ ]:
#sorting the dictionary

dd = sorted(d.items(), key=lambda x: x[1])

In [ ]:
len(dd)

In [ ]:
len(d)

In [ ]:
def in_edges(cat):
    n_cat = categories[cat]
    d = {}
    for n1 in n_cat:
        count = 0
        for n2 in graph:
            if n1 in graph[n2] and n2 in n_cat:
                count += 1
        d[n1] = count
    d = sorted(d.items(), key=lambda x: x[1])
    return d
    